In [1]:
import numpy as np

In [2]:
#defining an 9 x 9 cross word grid
grid = np.full((9,9),'#')
grid[0, 0:4] = '_' #Kiwi
grid[2, 1:6] = '_' #Melon
grid[8, 0:10] = '_' #Blueberry
grid[3:7, 0] = '_' #Plum
grid[2:7, 3] = '_' #Lemon
grid[0:9, 5] = '_' #Pineapple
grid[5:9, 7] = '_' #Pear
grid[0:5, 8] = '_' #Mango


#list of words
words = ["PINEAPPLE", "BLUEBERRY", "PLUM", "PEAR", "KIWI", "MANGO", "MELON", "LEMON"]
knowledge_base = {"PINEAPPLE" : ["0","5","V"], "BLUEBERRY": ["8","0","H"], "PLUM":["3","0","V"],
                  "PEAR":["5","7", "V"],"KIWI":["0","0","H"],"MANGO":["0","8","V"],"MELON":["2","1","H"],"LEMON":["2","3","V"]}

In [3]:
# Function to check if a word can be placed
def is_valid_placement(grid, word, row, col, direction):
    length = len(word)
    if direction not in ["H","V"]:
        return False
    placement = knowledge_base[word]
    if direction == "H":
        if col + length > len(grid[0]):
            return False
        if int(placement[0]) != row or placement[2] != direction:
            return False
        for i in range(length):
            if grid[row][col + i] != '_' and grid[row][col + i] != word[i]:
                return False
    elif direction == "V":
        if row + length > len(grid):
            return False
        if int(placement[1]) != col or placement[2] != direction:
            return False
        for i in range(length):
            if grid[row + i][col] != '_' and grid[row + i][col] != word[i]:
                return False
    return True           

In [4]:
# Function to place a word on the grid
def place_word(grid, word, row, col, direction):
    new_grid = np.copy(grid)
    length = len(word)
    if direction == "H":
        for i in range(length):
            new_grid[row][col + i] = word[i]
    elif direction == "V":
        for i in range(length):
            new_grid[row+i][col] = word[i]
    return new_grid

In [5]:
def evaluate(grid):
    score = 0
    for row in grid: 
        score += sum(1 for cell in row if cell != '_')
    return score

In [6]:
#Minmax algorithm with dept limit
def minmax(grid, words, depth, is_maximizing):   
    if depth == 0 or not words:
        return evaluate(grid), grid
    best_score = float('inf')    
    if is_maximizing:
        best_score = float('-inf')
    best_grid = None
    for word in words:
        for row in range(len(grid)):
            for col in range(len(grid[0])):
                for direction in ["H","V"]:
                    if is_valid_placement(grid, word, row, col, direction):
                        new_grid = place_word(grid, word, row, col, direction)
                        new_score, _ = minmax(new_grid, [w for w in words if w != word], depth - 1, not is_maximizing)
                        if is_maximizing:
                            if new_score > best_score:
                                best_score = new_score
                                best_grid = new_grid
                        else:
                            if new_score < best_score:
                                best_score = new_score
                                best_grid = new_grid
    return best_score, best_grid

In [7]:
#function to print the grid
def print_grid(grid):
    rowIndex = 1
    colIndexes = [' ',' ','1','2','3','4','5','6','7','8','9']
    print(" ".join(colIndexes))
    for row in grid:
        print(rowIndex, " ", " ".join(row))
        rowIndex += 1

In [8]:
def remove_ai_word_from_word_list():
    iteration_list = np.copy(words)
    
    for word in iteration_list:
        placement = knowledge_base[word]
        if placement[2] == "H":
            for row in grid:
                if word in ("".join(row)) and word in words:
                    words.remove(word)
        elif placement[2] == "V":
            for col in zip(*grid):
                if word in ("".join(col)) and word in words:
                    words.remove(word)
        

In [9]:
#Game with human input
def play_game():
    global grid, words
    human_score = 0
    ai_score = 0
    human_best_score = float('inf')
    while words:
        # Human Turn
        print("\n Current Grid: ")
        print_grid(grid)
        print("\n Available words: ", words)
        word = input("Enter a word to place: ").strip().upper()
        if word not in words:
            print("Invalid word! Try again.")
            continue
        row_input = input("Please enter a row number (1-9) to identify the cell where the word should start: ")
        if row_input.strip() == "" and not row_input.isnumeric() and  (int(row_input) < 1 or int(row_input) > 9):
            print("Invalid row! Try again.")
            
        col_input = input("Please enter a column number (1-9) to identify the cell where the word should start: ")
        if col_input.strip() == "" and not col_input.isnumeric() and  (int(col_input) < 1 or int(col_input) > 9):
            print("Invalid column! Try again.")
        
        row = int(row_input) - 1
        col = int(col_input) - 1
        direction = input("Please enter your direction. (H for horizontal and V for vertical) :").strip().upper()
        if not is_valid_placement(grid, word, row, col, direction):
            print("Invalid placement! Try again.")
            continue
        grid = place_word(grid, word, row, col, direction)
        score = len(word)
        if score > human_best_score:
            human_best_score = score
        human_score += score
        print_grid(grid)
        words.remove(word)

        # AI turn
        if words:
            print("\n AI is thinking ......")
            _, grid = minmax(grid, words, 3, True)
            words_before = set(np.copy(words))
            remove_ai_word_from_word_list()
            words_after = set(np.copy(words))
            ai_word = list(words_before - words_after)
            ai_score += len(ai_word[0])
            print("\n AI has placed a word. ", ai_word[0])           
            
            
    print("\n Final Score")
    print("Your score is ", human_score)
    if human_score > ai_score:
        print("You Won !!!!!  :)")
    else:
        print("You Lost !!!!! :(")
    print("\n Game Over !!!!!!!!!")

In [10]:
# Start the game
play_game()


 Current Grid: 
    1 2 3 4 5 6 7 8 9
1   _ _ _ _ # _ # # _
2   # # # # # _ # # _
3   # _ _ _ _ _ # # _
4   _ # # _ # _ # # _
5   _ # # _ # _ # # _
6   _ # # _ # _ # _ #
7   _ # # _ # _ # _ #
8   # # # # # _ # _ #
9   _ _ _ _ _ _ _ _ _

 Available words:  ['PINEAPPLE', 'BLUEBERRY', 'PLUM', 'PEAR', 'KIWI', 'MANGO', 'MELON', 'LEMON']


Enter a word to place:  kiwi
Please enter a row(1-9):  1
Please enter a column(1-9):  1
Please enter your direction. (H for horizontal and V for vertical) : h


    1 2 3 4 5 6 7 8 9
1   K I W I # _ # # _
2   # # # # # _ # # _
3   # _ _ _ _ _ # # _
4   _ # # _ # _ # # _
5   _ # # _ # _ # # _
6   _ # # _ # _ # _ #
7   _ # # _ # _ # _ #
8   # # # # # _ # _ #
9   _ _ _ _ _ _ _ _ _

 AI is thinking ......

 AI has placed a word.  PINEAPPLE

 Current Grid: 
    1 2 3 4 5 6 7 8 9
1   K I W I # P # # _
2   # # # # # I # # _
3   # _ _ _ _ N # # _
4   _ # # _ # E # # _
5   _ # # _ # A # # _
6   _ # # _ # P # _ #
7   _ # # _ # P # _ #
8   # # # # # L # _ #
9   _ _ _ _ _ E _ _ _

 Available words:  ['BLUEBERRY', 'PLUM', 'PEAR', 'MANGO', 'MELON', 'LEMON']


Enter a word to place:  blueberry
Please enter a row(1-9):  9
Please enter a column(1-9):  1
Please enter your direction. (H for horizontal and V for vertical) : h


    1 2 3 4 5 6 7 8 9
1   K I W I # P # # _
2   # # # # # I # # _
3   # _ _ _ _ N # # _
4   _ # # _ # E # # _
5   _ # # _ # A # # _
6   _ # # _ # P # _ #
7   _ # # _ # P # _ #
8   # # # # # L # _ #
9   B L U E B E R R Y

 AI is thinking ......

 AI has placed a word.  MANGO

 Current Grid: 
    1 2 3 4 5 6 7 8 9
1   K I W I # P # # M
2   # # # # # I # # A
3   # _ _ _ _ N # # N
4   _ # # _ # E # # G
5   _ # # _ # A # # O
6   _ # # _ # P # _ #
7   _ # # _ # P # _ #
8   # # # # # L # _ #
9   B L U E B E R R Y

 Available words:  ['PLUM', 'PEAR', 'MELON', 'LEMON']


Enter a word to place:  melon
Please enter a row(1-9):  3
Please enter a column(1-9):  2
Please enter your direction. (H for horizontal and V for vertical) : h


    1 2 3 4 5 6 7 8 9
1   K I W I # P # # M
2   # # # # # I # # A
3   # M E L O N # # N
4   _ # # _ # E # # G
5   _ # # _ # A # # O
6   _ # # _ # P # _ #
7   _ # # _ # P # _ #
8   # # # # # L # _ #
9   B L U E B E R R Y

 AI is thinking ......

 AI has placed a word.  PLUM

 Current Grid: 
    1 2 3 4 5 6 7 8 9
1   K I W I # P # # M
2   # # # # # I # # A
3   # M E L O N # # N
4   P # # _ # E # # G
5   L # # _ # A # # O
6   U # # _ # P # _ #
7   M # # _ # P # _ #
8   # # # # # L # _ #
9   B L U E B E R R Y

 Available words:  ['PEAR', 'LEMON']


Enter a word to place:  lemon
Please enter a row(1-9):  3
Please enter a column(1-9):  4
Please enter your direction. (H for horizontal and V for vertical) : v


    1 2 3 4 5 6 7 8 9
1   K I W I # P # # M
2   # # # # # I # # A
3   # M E L O N # # N
4   P # # E # E # # G
5   L # # M # A # # O
6   U # # O # P # _ #
7   M # # N # P # _ #
8   # # # # # L # _ #
9   B L U E B E R R Y

 AI is thinking ......

 AI has placed a word.  PEAR

 Final Score
Your score is  23
You Won !!!!!  :)

 Game Over !!!!!!!!!
